In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import cv2
import sys
import numpy as np
import glob
import pickle as pkl
from tqdm import tqdm
import random
import pandas as pd
from collections import OrderedDict

import keras
import keras.backend as K
from keras.models import model_from_json, Model, load_model
from keras.layers import Dense, Dropout
from keras.preprocessing import image as kpi

from sklearn.metrics import roc_auc_score, accuracy_score

import c3d_model

In [ ]:
LABEL_PICKLE = '../../data/rand_label_404_with_optical_entropy_all_emotions.pkl'
OPTICAL_FOLDER = './OpticalFlow/'
ENTROPY_FOLDER = './entropy/'
VIDEO_FOLDER = './video/'
C3D_MODEL_FILE = './models/sports1M_weights_tf.h5'

mean_cube = np.load('./models/train01_16_128_171_mean.npy')
mean_cube = np.transpose(mean_cube, (1, 2, 3, 0))

def create_c3d_model():
    c3d = c3d_model.get_model(backend='tf')
    c3d.load_weights(C3D_MODEL_FILE)
    last_dropout_name = [layer.name for layer in c3d.layers[::-1] if layer.name[:7] == 'dropout'][0]
    print(last_dropout_name)
    output = Dense(2, activation='softmax')(c3d.get_layer(last_dropout_name).output)
    model = Model(inputs=c3d.input, outputs=output)
    
    return model

def get_split_label(test_id):
    with open(LABEL_PICKLE, 'rb') as f:
        data = pkl.load(f)
    sub_folders = np.array_split(data, 5)
    test = sub_folders[test_id]
    train = [x for x in data if x not in test]
    return train, test

def get_single_image_data_from_label(label, emotion, etype, gt_type):
    X = np.zeros((len(label), 16, 112, 112, 3))
    Y = np.zeros((len(label), 2))
    for i, row in enumerate(tqdm(label)):
        name = str(int(row['Video ID'])).zfill(4)
        if etype == 'optical':
            image_path = glob.glob(OPTICAL_FOLDER+'{}_*.png'.format(name))
        elif etype == 'entropy':
            image_path = glob.glob(ENTROPY_FOLDER+'{}_*.png'.format(name))
#         elif etype == 'dnn':
#             image_path = glob.glob('/media/SeSaMe_NAS/share/sjfan/MMData404/DNNbase404/{}_*.jpg'.format(name)) 
        assert(len(image_path) == 1)
        image_path = image_path[0]
        image_data = get_image_data(image_path, mean_cube)
        X[i] = image_data
        if gt_type == 'optical' or gt_type == 'entropy':
            Y[i] = [1, 0] if row[gt_type+'_'+emotion] < 0.5 else [0, 1]
        elif gt_type == 'video':
            Y[i] = [1, 0] if row[emotion] < 0.5 else [0, 1]
    return X,Y

def get_video_data_from_label(label, emotion):
    X = np.zeros((len(label), 16, 112, 112, 3))
    Y = np.zeros((len(label), 2))
    for i, row in enumerate(tqdm(label)):
        name = str(int(row['Video ID'])).zfill(4)
        video_path = glob.glob(VIDEO_FOLDER+'{}_*.mp4'.format(name))
        assert(len(video_path) == 1)
        video_path = video_path[0]
        X[i] = get_video_data(video_path, mean_cube)
        Y[i] = [1, 0] if row[emotion] < 0.5 else [0, 1]
    return X,Y

# Create Model

In [ ]:
model = create_c3d_model()
model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])
model.summary()

In [ ]:
test_id = 0 # 0~4
batch_size = 16
epochs = 30
emotion = 'Valence'
vtype = 'video'

train_label, test_label = get_split_label(test_id)
if vtype == 'video':
    train_x, train_y = get_video_data_from_label(train_label, emotion)
elif vtype == 'entropy':
    train_x, train_y = get_single_image_data_from_label(train_label, emotion, 'entropy', gt_type='entropy')
elif vtype == 'optical':
    train_x, train_y = get_single_image_data_from_label(train_label, emotion, 'optical', gt_type='optical')

model.fit(train_x, train_y, batch_size=batch_size,epochs=epochs, 
              verbose=1)